# 基础检索增强生成（RAG）

此代码实现了一个基本的检索增强生成（RAG）系统，用于处理和查询PDF文档。系统将文档内容编码到矢量存储中，然后可以查询该矢量存储以检索相关信息。

### 关键步骤
1. 处理PDF文档并提取文本
2. 对文本内容进行分块
3. 使用Qdrant创建向量数据和minimax Embedding进行存储向量数据
4. 创建检索器进行查询向量数据

导入库和设置环境变量

In [36]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["MINIMAX_GROUP_ID"] = os.getenv("MINIMAX_GROUP_ID")
os.environ["MINIMAX_API_KEY"] = os.getenv("MINIMAX_API_KEY")

from langchain_qdrant import QdrantVectorStore
from langchain_community.embeddings import MiniMaxEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

#### 第一步 处理PDF文档和提取文本内容
加载PDF文档采用PyPDF库进行文档加载，需要提前加载“pip install pypdf”,并且使用from langchain_community.document_loaders import PyPDFLoader这个类。

In [37]:
path = "./data/问答题.pdf"
loader = PyPDFLoader(path)
documents = loader.load()

#### 第二步 对提取的文本进行分块
使用langchain的split_text方法，将文本进行分块，方便后续的检索和生成

In [38]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len
)
docs = text_splitter.split_documents(documents)
print(f"执行 递归字符 切割文档，切割{len(docs)}块")


执行 递归字符 切割文档，切割25块


#### 第三步 创建embeddings和向量数据库
使用本地搭建的Qdrant数据库

In [39]:
embeddings = MiniMaxEmbeddings()
vectorstore = QdrantVectorStore.from_documents(
    docs,
    embeddings,
    url="http://localhost:6333/",
    prefer_grpc=True,
    collection_name="qa"
)

#### 第四步 创建retriever检索器

In [40]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [41]:
query = "存费送权益我能参加吗"
result = retriever.invoke(query)
result

[Document(metadata={'page': 3, 'source': './data/问答题.pdf', '_id': '1539a3c4-cdcd-4e5a-b644-e0b81d053558', '_collection_name': 'qa'}, page_content='问：存费送权益的的权益包是什么\n答：⼤权益包A包：以下权益每⽉N 选2：爱奇艺周卡/优酷周卡/腾讯周卡/5元⽔ 电煤缴费券/5元中⽯油\n加油券/5元商户券  \n⼤权益包B包：以下权益每⽉N 选1：爱奇艺⽉卡/优酷⽉卡/腾讯⽉卡/1 5元⽔电煤缴费券（5元* 3张）/1 5\n元中⽯油元券/15元商户券（5元*3张）  \n⼤权益包在总价值不减少的前提下，包含的内容可能会不定期进⾏更新调整，具体⻅翼⽀付客户端权益\n专区展示内容。\n问：存费送权益的话费怎么返还和使⽤呢\n答：当⽉起逐⽉返还话费，⾸⽉返还话费办理后24⼩时内到账，后续每⽉1⽇返还话费。返还话费可抵扣'),
 Document(metadata={'page': 3, 'source': './data/问答题.pdf', '_id': 'fcfa2245-83e9-4ad7-8989-7aef5ddf3382', '_collection_name': 'qa'}, page_content='4问：存费送权益我能参加吗\n答：您好，⽬前这个活动针对安徽电信翼⽀付预付费、后付费⼿机⽤户均可办理\n问：存费送权益的活动内容是什么样的\n答：活动1：预存300元，协议期6个⽉，分6个⽉返还话费，每个⽉返50元话费，并赠送⼤权益包A包。  \n活动2：预存600元，协议期12个⽉，分12个⽉返还话费，每个⽉返50元话费，并赠送⼤权益包B包。  \n问：存费送权益的的权益包是什么\n答：⼤权益包A包：以下权益每⽉N 选2：爱奇艺周卡/优酷周卡/腾讯周卡/5元⽔ 电煤缴费券/5元中⽯油\n加油券/5元商户券')]